<a href="https://colab.research.google.com/github/sureshkasiraboina/sureshkasira/blob/master/User_Attribute_Mapping_Attribute_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install looker-sdk

!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import looker_sdk
import requests
import pandas as pd
from looker_sdk import  models40
import json

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Initialize Looker SDK using the configuration file path
sdk = looker_sdk.init40("/content/drive/My Drive/credintial.ini")

# Get information about the authenticated user
user = sdk.me()

# # Print the user's ID
print(f"User ID: {user.id}")

# # Print the user's display name
print(f"User Name: {user.display_name}")

**Upload the excel file Colab Folder and Read the file**

In [ ]:
# Assuming your file is named 'file.xlsx'
df = pd.read_excel('/content/engagementrls.xlsx')

# Display the DataFrame
df

**GETTING USER ID from USER EMAIL**

In [ ]:
def get_user_id_by_email(email):
    try:
        user = sdk.search_users(email=email)
        if user:
            return user[0].id
        else:
            return None
    except Exception as e:
        print(f"Error retrieving user ID for email {email}: {e}")
        return None

def add_user_ids_to_df(df):
    # Check if DataFrame has the necessary columns
    if 'Email' not in df.columns:
        raise KeyError("The DataFrame must contain an 'Email' column.")

    # Get unique emails from the DataFrame
    unique_emails = df['Email'].unique()

    # Create a dictionary to store email to user_id mapping
    email_to_user_id = {}

    # Fetch user IDs for each unique email
    for email in unique_emails:
        user_id = get_user_id_by_email(email)
        email_to_user_id[email] = user_id

    # Map user IDs back to the DataFrame
    df['user_id'] = df['Email'].map(email_to_user_id)

    return df
df = add_user_ids_to_df(df)
df

**ADDING A COLUMN NAMED ATTRIBUTE_ID and changing column name**

In [ ]:
df['attribute_id'] = 47
df = df.rename(columns={'School': 'attribute_value'})
df

**Assigning Attribute Values to the Users**

In [ ]:


def set_user_attribute_values(df):
    # Check if DataFrame has the necessary columns
    if not {'user_id', 'attribute_id', 'attribute_value'}.issubset(df.columns):
        raise KeyError("The DataFrame must contain 'user_id', 'attribute_id', and 'attribute_value' columns.")

    # Group by user_id and attribute_id, then aggregate the attribute_values
    aggregated_df = df.groupby(['user_id', 'attribute_id'])['attribute_value'].apply(lambda x: ','.join(x)).reset_index()

    # Iterate through each row in the aggregated DataFrame
    for index, row in aggregated_df.iterrows():
        user_id = row['user_id']
        attribute_id = str(row['attribute_id'])  # Convert to string if required
        attribute_value = row['attribute_value']

        # Print for debugging
        print(f"Updating user_id: {user_id}, attribute_id: {attribute_id}, value: {attribute_value}")

        # Update the user attribute using Looker SDK
        try:
            # Ensure the API method and parameters match the documentation
            sdk.set_user_attribute_user_value(
                user_id=user_id,  # Ensure this is an integer
                user_attribute_id=attribute_id,  # Convert this if needed
                body={'value': attribute_value}  # Ensure this is a dictionary
            )
            print(f"Updated user_id {user_id} with attribute_id {attribute_id} to value {attribute_value}")
        except Exception as e:
            print(f"Error updating user_id {user_id} with attribute_id {attribute_id}: {e}")


set_user_attribute_values(df)